In [1]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [2]:
import os
import zipfile
import numpy as np
from scipy.stats import pearsonr 
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, layers
from keras import callbacks
from keras import optimizers, losses
from sklearn.metrics import roc_auc_score

In [3]:
traindir = "/kaggle/input/psychosis-classification-with-rsfmri/data/train"
classes = os.listdir(traindir)
print(classes)

['BP', 'SZ']


In [4]:
BPdir = os.path.join(traindir, 'BP')
SZdir = os.path.join(traindir, 'SZ')
BPfiles = os.listdir(BPdir)
print('Total BP samples =', len(BPfiles))
SZfiles = os.listdir(SZdir)
print('Total SZ samples =', len(SZfiles))

# randomly taken a sample of a subject
person = os.path.join(SZdir, SZfiles[54])
ICN_path = os.path.join(person, 'icn_tc.npy') 
icn_data = np.load(ICN_path)
ROI = icn_data.shape[1] 

Total BP samples = 183
Total SZ samples = 288


In [5]:
testdir = "/kaggle/input/psychosis-classification-with-rsfmri/data/test"
testfiles = os.listdir(testdir)
print(len(testfiles))

315


In [6]:
X = np.zeros((len(BPfiles)+len(SZfiles), ROI, ROI, 3))
print(X.shape) 
print(X[0, :, :, 1].shape)

(471, 105, 105, 3)
(105, 105)


In [7]:
for k in range(len(BPfiles)):

    person = os.path.join(BPdir, BPfiles[k])
    ICN_path = os.path.join(person, 'icn_tc.npy') 
    icn_data = np.load(ICN_path)
    X[k, :, :, 0] = np.corrcoef(icn_data, rowvar=False) 
    X[k, :, :, 1] = np.corrcoef(icn_data, rowvar=False) 
    X[k, :, :, 2] = np.corrcoef(icn_data, rowvar=False) 

In [8]:
for k in range(len(SZfiles)):
    person = os.path.join(SZdir, SZfiles[k])
    ICN_path = os.path.join(person, 'icn_tc.npy') 
    icn_data = np.load(ICN_path)
    X[k + len(BPfiles), :, :, 0] = np.corrcoef(icn_data, rowvar=False) 
    X[k + len(BPfiles), :, :, 1] = np.corrcoef(icn_data, rowvar=False) 
    X[k + len(BPfiles), :, :, 2] = np.corrcoef(icn_data, rowvar=False) 

In [9]:
X_test = np.zeros((len(testfiles), ROI, ROI, 3))
print(X_test.shape) 
print(X_test[0, :, :, 1].shape)

(315, 105, 105, 3)
(105, 105)


In [10]:
for k in range(len(testfiles)):
    person = os.path.join(testdir, testfiles[k])
    ICN_path = os.path.join(person, 'icn_tc.npy') 
    icn_data = np.load(ICN_path)
    X_test[k, :, :, 0] = np.corrcoef(icn_data, rowvar=False) 
    X_test[k, :, :, 1] = np.corrcoef(icn_data, rowvar=False)
    X_test[k, :, :, 2] = np.corrcoef(icn_data, rowvar=False)

In [11]:
print(X_test.shape)
print(X_test[0, :, :, 2])

(315, 105, 105, 3)
[[ 1.          0.03753258 -0.01587203 ...  0.07886888 -0.02934984
  -0.09429664]
 [ 0.03753258  1.          0.18835413 ...  0.09759987  0.32896269
   0.54525875]
 [-0.01587203  0.18835413  1.         ... -0.30397956 -0.09328281
   0.27499341]
 ...
 [ 0.07886888  0.09759987 -0.30397956 ...  1.          0.40946951
   0.06334757]
 [-0.02934984  0.32896269 -0.09328281 ...  0.40946951  1.
   0.56887192]
 [-0.09429664  0.54525875  0.27499341 ...  0.06334757  0.56887192
   1.        ]]


In [12]:
BP_target = np.ones(len(BPfiles))
SZ_target = np.zeros(len(SZfiles)) 
Y = np.concatenate((BP_target, SZ_target)) 
Y = Y.reshape((-1, 1))
print(Y.shape) 

(471, 1)


In [13]:
! pip install -U git+https://github.com/awsaf49/efficientnet-spec

  Cloning https://github.com/awsaf49/efficientnet-spec to /tmp/pip-req-build-275_peco
  Running command git clone --filter=blob:none --quiet https://github.com/awsaf49/efficientnet-spec /tmp/pip-req-build-275_peco
  Resolved https://github.com/awsaf49/efficientnet-spec to commit 82c166d368464aee8ca0ec240d32c85aad16713d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00
  Created wheel for efficientnet: filename=efficientnet-1.1.1-py3-none-any.whl size=22593 sha256=415a86f31276a3f836d2bea2ca2431abfbd602a10be873a64ecb09ab1fc3b96f
  Stored in directory: /tmp/pip-ephem-wheel-cache-j7lx2901/wheels/60/e2/9e/10f1b7cac06e056e776d8f9a607fca1c9629e6e1a75205a587
Successfully built efficientnet


In [14]:
import efficientnet.keras as efn 

def model_definition(ROI):
    input = keras.Input(shape=(ROI, ROI, 3))

    base_model = efn.EfficientNetB0(
            weights="imagenet",  
            input_shape=(ROI, ROI, 3),
            include_top=False,
        )  

    base_model.trainable = True

    x = base_model(input)
    x = layers.GlobalAveragePooling2D()(x)
    # # x = layers.Dropout(0.2)(x)
    # x = layers.Flatten()(x)
    x = layers.Dense(20, activation = 'relu')(x)
    output = layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.Model(input, output)

    return model

model = model_definition(ROI)
print(model.summary()) 

16804768/16804768 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 105, 105, 3)]     0         
                                                                 
 efficientnet-b0 (Functional  (None, 7, 7, 1280)       4049564   
 )                                                               
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 20)                25620     
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                              

In [15]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, mode='min')

In [16]:
from sklearn.model_selection import KFold

acc_per_fold = []
loss_per_fold = []
roc_scores = []
test_predictions = []
kfold = KFold(n_splits=10, shuffle=True, random_state = 2)


fold_no = 1
for train, test in kfold.split(X, Y):
    
    # Defining the model 
    model = model_definition(ROI)

    
    # Compile the model
    model.compile(loss=losses.BinaryCrossentropy(), 
              optimizer=optimizers.Adam(), 
              metrics=['accuracy'])  

    
    
    # Fit data to model
    history = model.fit(X[train], Y[train],
                batch_size=int(np.ceil(len(train)/3)),
                epochs=50, 
                verbose=0, 
                callbacks=[callback])

    
    # Generate generalization metrics
    true = list(Y[test])
    pred = list(model.predict(X[test]).reshape(-1))
    roc = roc_auc_score(np.array(true), np.array(pred))

    if roc > 0.65:
      prediction = list(model.predict(X_test).reshape(-1))
      test_predictions.append(prediction)

    roc_scores.append(roc)
    
    scores = model.evaluate(X[test], Y[test])
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1


# == Provide average scores ==
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


2023-03-08 04:25:26.153616: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/efficientnet-b0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2/2 [==============================] - 2s 25ms/step - loss: 0.9497 - accuracy: 0.6667


2023-03-08 04:26:21.296306: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/efficientnet-b0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2/2 [==============================] - 1s 24ms/step - loss: 3.8629 - accuracy: 0.5745


2023-03-08 04:27:17.963641: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_3/efficientnet-b0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


...

2/2 [==============================] - 1s 25ms/step - loss: 5.2255 - accuracy: 0.6596


2023-03-08 04:32:11.185587: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_9/efficientnet-b0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2/2 [==============================] - 2s 25ms/step - loss: 0.9646 - accuracy: 0.4681


2023-03-08 04:33:03.696721: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_10/efficientnet-b0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2/2 [==============================] - 3s 25ms/step - loss: 1.9671 - accuracy: 0.4681
Average scores for all folds:
> Accuracy: 54.32624101638794 (+- 10.770805132466016)
> Loss: 2.6281534910202025
------------------------------------------------------------------------


In [17]:
print(np.average(roc_scores))

0.5222139656548933


In [18]:
y_test_pred = np.array(test_predictions)
print(type(y_test_pred))
print(y_test_pred.shape)

<class 'numpy.ndarray'>
(1, 315)
